In [96]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score
import yfinance as yf

In [97]:
# Load your dataset
df_original = pd.read_excel("Ellis_Final_Data.xlsx")

# Convert 'date' to datetime format
df_original['date'] = pd.to_datetime(df_original['date'])

# Extract year for grouping
df_original['year'] = df_original['date'].dt.year

# Function to assign categories based on percentiles
def assign_category(df):
    top_thresh = df['Forward 3-Month Ret'].quantile(0.85)
    bottom_thresh = df['Forward 3-Month Ret'].quantile(0.15)
    
    df['Rating'] = 'medium'
    df.loc[df['Forward 3-Month Ret'] >= top_thresh, 'Rating'] = 'winner'
    df.loc[df['Forward 3-Month Ret'] <= bottom_thresh, 'Rating'] = 'loser'
    
    return df

# Apply function year-wise
df = df_original.groupby('year', group_keys=False).apply(assign_category)

/var/folders/q_/bgcxlv5x1n3723c2hpfljwl80000gn/T/ipykernel_8847/2384310978.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df_original.groupby('year', group_keys=False).apply(assign_category)


In [98]:
# Convert 'Rating' to numerical values (assuming 'winner', 'medium', 'loser' as categories)
df['Rating'] = df['Rating'].map({'winner': 1, 'medium': 0, 'loser': -1})

# Drop the 'Forward 3-Month Ret' columns
df = df.drop(columns=['Forward 3-Month Ret'])

In [99]:
# Store model performance and feature importance
model_performance = []
feature_importance = {}

# Drop 'date' and 'ticker' from features but keep them for filtering
target = "Rating"

# Collect all possible features before training
all_features = set()

# Prepare train data with windoews of 5 years and encode categorical variables
for start_year in range(2003, 2019):  # Last start year: 2018
    train_data = df[(df['date'].dt.year >= start_year) & (df['date'].dt.year <= start_year + 4) &
                    (df['date'].dt.month >= 1) & (df['date'].dt.month <= 9)]
    train_data = pd.get_dummies(train_data, columns=['Sector_name'], drop_first=True)
    all_features.update(train_data.columns)

# Remove target and identifier columns
all_features -= {target, "date", "TICKER"}
all_features = sorted(all_features)

# Rolling window training and validation
model_predictions = {}
model_weights = {}

# Run model for each window
for start_year in range(2003, 2019):  # Last start year: 2018
    train_start = f"{start_year}-01"
    train_end = f"{start_year + 4}-09"
    
    val_dates = [
        f"{start_year + 4}-12",
        f"{start_year + 5}-03",
        f"{start_year + 5}-06",
        f"{start_year + 5}-09",
    ]
    
    # Training data
    train_data = df[(df['date'].dt.year >= start_year) & (df['date'].dt.year <= start_year + 4) &
                    (df['date'].dt.month >= 1) & (df['date'].dt.month <= 9)]
    train_data = pd.get_dummies(train_data, columns=['Sector_name'], drop_first=True)
    train_data = train_data.reindex(columns=all_features, fill_value=0)
    X_train = train_data
    y_train = df.loc[train_data.index, target]
    
    # Train model
    model = LogisticRegression(max_iter=1000, penalty='l1', solver='liblinear',class_weight='balanced',random_state=42)
    model.fit(X_train, y_train)
    
    # Store feature importance
    feature_importance[f"Train {train_start} - {train_end}"] = {
        feature: abs(coef) for feature, coef in zip(all_features, model.coef_.flatten())
    }
    
    # Validate model
    val_f1_scores = []
    for val_date in val_dates:
        val_data = df[df['date'].dt.strftime('%Y-%m') == val_date]
        if not val_data.empty:
            val_data = pd.get_dummies(val_data, columns=['Sector_name'], drop_first=True)
            X_val = val_data.reindex(columns=all_features, fill_value=0)
            y_val = df.loc[val_data.index, target]
            if len(y_val.unique()) > 1:
                preds = model.predict(X_val)
                f1 = f1_score(y_val, preds, average='weighted')
                val_f1_scores.append(f1)
            else:
                val_f1_scores.append(np.nan)
        else:
            val_f1_scores.append(np.nan)
    
    mean_f1 = np.nanmean(val_f1_scores) if len(val_f1_scores) > 0 else 0
    model_performance.append({'Start Year': start_year, 'Validation F1 Scores': val_f1_scores, 'Mean F1': mean_f1})
    model_weights[start_year] = mean_f1  # Store mean F1 for weighting
    
    # Predict December 2023
    dec_2023_data = df[df['date'].dt.strftime('%Y-%m') == '2023-12']
    if not dec_2023_data.empty:
        dec_2023_data = pd.get_dummies(dec_2023_data, columns=['Sector_name'], drop_first=True)
        X_dec_2023 = dec_2023_data.reindex(columns=all_features, fill_value=0)
        model_predictions[start_year] = model.predict(X_dec_2023)

# Normalize weights
total_weight = sum(model_weights.values())
normalized_weights = {year: (weight / total_weight) for year, weight in model_weights.items()} if total_weight > 0 else None

# Compute weighted average predictions
if normalized_weights:
    weighted_preds = sum(
        model_predictions[year] * normalized_weights[year] for year in model_predictions.keys()
    )
    final_dec_2023_preds = np.round(weighted_preds)  # Convert to class labels
else:
    final_dec_2023_preds = np.mean(list(model_predictions.values()), axis=0)
    final_dec_2023_preds = np.round(final_dec_2023_preds)

# Convert results to DataFrames
performance_df = pd.DataFrame(model_performance)
feature_importance_df = pd.DataFrame(feature_importance).fillna(0)

dec_2023_results = pd.DataFrame({
    'TICKER': df[df['date'].dt.strftime('%Y-%m') == '2023-12']['TICKER'],
    'Predicted_Class': final_dec_2023_preds
})

In [ ]:
# Extract data from yahoo finance for the predicted winners
stocks = dec_2023_results[dec_2023_results["Predicted_Class"] == 1]['TICKER']  # Select the 'TICKER' column
returns_data = {}

for stock in stocks:
    try:
        # Keep the original ticker for saving later
        original_stock = stock
        
        # Replace GPS with GAP for fetching data
        if stock == "GPS":
            stock = "GAP"
        
        # Replace TUP with TUP.HM for fetching data
        if stock == "TUP":
            stock = "TUP.HM"

        ticker = yf.Ticker(stock)
        hist = ticker.history(start="2023-12-01", end="2024-03-31")
        
        if hist.empty:
            print(f"No data found for {stock}. Skipping...")  # Debugging print
            continue

        # Filter dates
        hist_dec = hist[(hist.index >= "2023-12-01") & (hist.index <= "2023-12-31")]
        hist_mar = hist[(hist.index >= "2024-03-01") & (hist.index <= "2024-03-31")]

        if hist_dec.empty or hist_mar.empty:
            print(f"Missing data for {stock} in December 2023 or March 2024. Skipping...")  # Debugging print
            continue

        # Calculate average price
        dec_2023_price = hist_dec['Close'].mean() if not hist_dec.empty else None
        mar_2024_price = hist_mar['Close'].mean() if not hist_mar.empty else None

        # Compute forward return
        if dec_2023_price and mar_2024_price:
            forward_return = (mar_2024_price - dec_2023_price) / dec_2023_price
            returns_data[original_stock] = forward_return  # Save the original ticker
        else:
            print(f"Data missing for {stock} in either December 2023 or March 2024. Skipping...")  # Debugging print

    except Exception as e:
        print(f"Error fetching data for {stock}: {e}")

# Convert to DataFrame
returns_df = pd.DataFrame(returns_data.items(), columns=['TICKER', 'Forward 3-Month Ret'])

$MRO: possibly delisted; no timezone found


No data found for MRO. Skipping...


In [ ]:
# Extract data from yahoo finance for the S&P500 market index
try:
    # Fetch the S&P 500 data
    sp500 = yf.Ticker("^GSPC")
    sp500_hist = sp500.history(start="2023-12-01", end="2024-03-31")  # Define the date range

    if not sp500_hist.empty:
        # Filter the data for December 2023 and March 2024
        sp500_dec = sp500_hist[(sp500_hist.index >= "2023-12-01") & (sp500_hist.index <= "2023-12-31")]
        sp500_mar = sp500_hist[(sp500_hist.index >= "2024-03-01") & (sp500_hist.index <= "2024-03-31")]

        # Calculate the average price for each period
        sp500_dec_2023 = sp500_dec['Close'].mean() if not sp500_dec.empty else None
        sp500_mar_2024 = sp500_mar['Close'].mean() if not sp500_mar.empty else None

        # Compute forward return (if both prices are available)
        if sp500_dec_2023 and sp500_mar_2024:
            sp500_forward_return = (sp500_mar_2024 - sp500_dec_2023) / sp500_dec_2023
        else:
            sp500_forward_return = None
    else:
        sp500_forward_return = None
except Exception as e:
    print(f"Error fetching S&P 500 data: {e}")
    sp500_forward_return = None

# Add S&P 500 return as a constant column to your DataFrame
df["SP500 Forward 3-Month Ret"] = sp500_forward_return

In [102]:
# Define output file name
output_file = "Ellis_Final_Output.xlsx"

# Filter the "1" class predicted stocks
predicted_winners = dec_2023_results[dec_2023_results["Predicted_Class"] == 1]

# Randomly select up to 40 stocks
random_winners = predicted_winners.sample(n=min(40, len(predicted_winners)), random_state=43)

# Add the S&P 500 forward return to the random winners DataFrame
random_winners["SP500 Forward 3-Month Ret"] = sp500_forward_return  # Add the constant S&P 500 return to each row

# Merge the returns_df with the random_winners based on TICKER
random_winners_with_returns = random_winners.merge(returns_df, on="TICKER", how="left")

# Create an Excel writer with multiple sheets
with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    # Save Predictions to the first sheet
    dec_2023_results[["TICKER", "Predicted_Class"]].to_excel(writer, sheet_name="Predictions", index=False)
    
    # Save Feature Importance to the second sheet
    feature_importance_df.to_excel(writer, sheet_name="Feature Importance")

    # Save Random 1-Class Stocks to the third sheet, now including 'Forward 3-Month Ret'
    random_winners_with_returns[["TICKER", "Predicted_Class", "SP500 Forward 3-Month Ret", "Forward 3-Month Ret"]].to_excel(writer, sheet_name="Final Portfolio", index=False)